### 데이터 셋 나누기

In [ ]:
import numpy as np
import torch
import torch.optim as optim
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)


def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_u):
    squared_diffs = (t_p - t_u) ** 2
    return squared_diffs.mean()


- 인덱스 텐서 생성

In [ ]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples) #torch.randperm(n) n 범위 내에서 무작위 순열 생성

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
print('n_samples : ', n_samples, 'n_val :' , n_val ,'shuffled_indices :', shuffled_indices)
print(n_val)
print(shuffled_indices)
print('train_indices : \n', train_indices)
print('val_indices : \n' , val_indices)

- 훈련셋 , 검증셋 생성

In [ ]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = train_t_u * 0.1
val_t_un = val_t_u * 0.1

In [ ]:
def training_loop(n_epochs , optimizer , params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1,n_epochs+1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p,train_t_c)

        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <=3 or epoch %  500 == 0:
            print(f"Epoch : {epoch}  , Training Loss : {train_loss.item():.4f} , \n Validation Loss  : {val_loss.item():.4f}")

    return params




In [ ]:
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)

In [ ]:
training_loop(n_epochs = 3000, 
              optimizer = optimizer, 
              params = params, 
              train_t_u = train_t_un, 
              val_t_u = val_t_un, 
              train_t_c = train_t_c, 
              val_t_c = val_t_c)